# 3 Advanced Chains in LangChain
## Topics
* 3.1 Parallel Chains
* 3.2 Branching Chains
* 3.3 Output Parsers
* 3.4 Memory in Chains
* 3.5 Practice Task

In [2]:
import os
import sys
from pathlib import Path

sys.path.append(os.path.abspath(".."))

In [3]:
# initializing the llm
from llm.load_llm import initialize_llm

llm = initialize_llm()

LLM ready: ChatGoogleGenerativeAI


In [4]:
## Importing necessary modules
from langchain.chains import LLMChain, SequentialChain, SimpleSequentialChain, ConversationChain
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

### 3.1 Parallel Chains
Goal: for one input (topic) produce both a summary and a quiz question.
We show two simple approaches: (A) run chains one after the other and collect results (simple & clear), (B) optional: do them concurrently using ThreadPoolExecutor (speedup if you want parallel HTTP calls).

#### 3.1.A Simple/Sequential Collection

In [5]:
summary_prompt = PromptTemplate.from_template("Wrtie a short story (1-2 sentence) summary about {topic}")
quiz_prompt = PromptTemplate.from_template("Write one quiz question about {topic} with correct answer on a new line labeled 'Answer:'")

## chains
summary_chain = LLMChain(
    llm = llm,
    prompt = summary_prompt
)
quiz_chain = LLMChain(
    llm = llm,
    prompt = quiz_prompt
)
# run them and collect results
topic = "Finite Automata"
summary = summary_chain.invoke({"topic":topic})
quiz = quiz_chain.invoke({"topic":topic})

result = {"topic": topic, "summary": summary, "quiz": quiz}
print(result)

C:\Users\amalv\AppData\Local\Temp\ipykernel_18212\343937080.py:5: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  summary_chain = LLMChain(


{'topic': 'Finite Automata', 'summary': {'topic': 'Finite Automata', 'text': 'A diligent but simple machine, the Finite Automaton, follows a strict set of rules, reading input one symbol at a time and changing its state accordingly, ultimately accepting or rejecting the input based on its final state, much like a highly specialized vending machine only dispensing treats for specific code sequences.'}, 'quiz': {'topic': 'Finite Automata', 'text': 'Which of the following statements is TRUE regarding a Deterministic Finite Automaton (DFA)?\n\nA) A DFA can have multiple transitions from a single state on the same input symbol.\nB) A DFA can accept an infinite number of strings.\nC) A DFA always halts on every input string.\nD) A DFA can have transitions that lead to no state (a "dead end").\n\n\nAnswer: B'}}


In [11]:
import json

print(json.dumps(result , indent=2))

{
  "topic": "Finite Automata",
  "summary": {
    "topic": "Finite Automata",
    "text": "A diligent but simple machine, the Finite Automaton, follows a strict set of rules, reading input one symbol at a time and changing its state accordingly, ultimately accepting or rejecting the input based on its final state, much like a highly specialized vending machine only dispensing treats for specific code sequences."
  },
  "quiz": {
    "topic": "Finite Automata",
    "text": "Which of the following statements is TRUE regarding a Deterministic Finite Automaton (DFA)?\n\nA) A DFA can have multiple transitions from a single state on the same input symbol.\nB) A DFA can accept an infinite number of strings.\nC) A DFA always halts on every input string.\nD) A DFA can have transitions that lead to no state (a \"dead end\").\n\n\nAnswer: B"
  }
}


### 3.1.B Run in parallel(concurrent calls)
* Runs chains concurrently. Useful when we want speed
* thread-based concurrency calls both chains at the same time. Be mindful of rate limits and thread-safety for your environment.

`with ThreadPoolExecutor(max_workers=2) as ex:`
- Makes a pool of 2 threads (workers) that can do tasks at the same time.
- ex is the executor that controls them.

``` python
futures = {}
    for name, chain in chains:
        future = ex.submit(run_chain , chain , inp)
        futures[future] = name
````
* For each chain:
    - ex.submit(run_chain, chain, inp) schedules the function run_chain(chain, inp) to run in a thread.
    - Returns a Future object (a placeholder for the result).
    - We store it in futures, mapping the future to the chain’s name.

``` python
parallel_result = {}
for future, name in futures.items():
    parallel_result[name] = future.result()
```
- Loop through each stored future.
- future.result() waits for the thread to finish and returns the actual output from the chain.
- Store it in a dictionary with the chain’s name as the key.

In [13]:
from concurrent.futures import ThreadPoolExecutor

def run_chain(chain , inp):
    return chain.invoke(inp)

chains = [
          ("summary" , summary_chain),
          ("quiz" , quiz_chain)
         ]
inp = {"topic":"Reinforcement learning"}

with ThreadPoolExecutor(max_workers=2) as ex:
    futures = {}
    for name, chain in chains:
        future = ex.submit(run_chain , chain , inp)
        futures[future] = name
    
    parallel_result = {}
    for future, name in futures.items():
        parallel_result[name] = future.result()

print(json.dumps(parallel_result, indent=2))

{
  "summary": {
    "topic": "Reinforcement learning",
    "text": "A resourceful robot, initially clumsy, learns to navigate a complex maze by trial and error, receiving rewards for successful moves and penalties for failures, gradually perfecting its path through repeated attempts and adapting its strategy based on the consequences of its actions.  This process, refined through countless iterations, exemplifies the core principles of reinforcement learning."
  },
  "quiz": {
    "topic": "Reinforcement learning",
    "text": "Which of the following is NOT a core component of a Reinforcement Learning agent?\n\nA) Policy\nB) Reward function\nC) Value function\nD) Supervised learning model\n\n\nAnswer: D) Supervised learning model"
  }
}


### 3.2 — Branching Chains (conditional flows)

Goal: Inspect user input (or a classifier chain) and pick one of multiple chains to run.

We’ll:
- Use a tiny classifier prompt to pick a route (math, story, or general).
- Based on the result run the appropriate chain.

In [15]:
# Route chains
math_prompt = PromptTemplate.from_template("Solve the math question: {q} and give the final numeric answer only.")
math_chain = LLMChain(llm=llm, prompt=math_prompt)

story_prompt = PromptTemplate.from_template("Write a short (3-sentence) fictional scene for this prompt: {q}")
story_chain = LLMChain(llm=llm, prompt=story_prompt)

general_prompt = PromptTemplate.from_template("Answer the question concisely: {q}")
general_chain = LLMChain(llm=llm, prompt=general_prompt)

# Classifier: ask the model to decide which type it is
classifier_prompt = PromptTemplate.from_template(
    "Classify the following user request into ONE of these single-word categories: math, story, general.\n\nRequest: \"{q}\"\n\nAnswer with exactly one word (math, story, or general)."
)
classifier_chain = LLMChain(llm=llm, prompt=classifier_prompt)

def route_and_run(q):
    cat = classifier_chain.invoke({"q": q})["text"].lower()
    # normalize
    cat = cat.split()[0] if cat else "general"
    if "math" in cat:
        return {"route": "math", "output": math_chain.invoke({"q": q})}
    elif "story" in cat:
        return {"route": "story", "output": story_chain.invoke({"q": q})}
    else:
        return {"route": "general", "output": general_chain.invoke({"q": q})}

# Try a math prompt
print(route_and_run("Solve 12 * 7 + 5"))

# Try a story prompt
print(route_and_run("Write a cozy scene about a lost map found in an attic"))


{'route': 'math', 'output': {'q': 'Solve 12 * 7 + 5', 'text': '94'}}
{'route': 'story', 'output': {'q': 'Write a cozy scene about a lost map found in an attic', 'text': "Dust motes danced in the lone sunbeam slicing through the attic's gloom as Elara unearthed a rolled parchment.  Unfurling it carefully, she gasped – a hand-drawn map, faded but vibrant, depicting a winding path leading to a heart-shaped lake nestled amongst rolling hills.  The scent of old paper and forgotten adventures filled her senses, promising a journey far beyond the attic walls."}}


this manual routing pattern works well and is transparent for debugging. For production you can make the classifier more strict or use deterministic heuristics.

### 3.3 - Output Parsers (JSON structured ouptuts)

Ask the LLM to return JSON, then parse it into structured data

#### 3.3.A Use JSON

In [20]:
import json

json_prompt = PromptTemplate.from_template(
    """You are an assistant that returns strict JSON only. For a product named '{product}', return a JSON object with keys:\n"
    Return the result ONLY in valid JSON with this format:
    {{
      "name": "...",
      "tagline": "..."
    }}"""
)

json_chain = LLMChain(llm=llm, prompt=json_prompt)

response_text = json_chain.invoke({"product": "eco-friendly water bottle"})
print("Raw response:\n", response_text["text"])

# Try to parse
try:
    parsed = json.loads(response_text["text"])
    print("Parsed JSON:", parsed)
except json.JSONDecodeError:
    print("JSON parsing failed. Response was not valid JSON. Consider asking model to return valid JSON or use a stricter parser.")


Raw response:
 ```json
{
  "name": "eco-friendly water bottle",
  "tagline": "Hydrate sustainably."
}
```
JSON parsing failed. Response was not valid JSON. Consider asking model to return valid JSON or use a stricter parser.


#### 3.3.B Use LangChains built-in JSON parser
LangChain has a tool called StructuredOutputParser that ensures proper

`ResponseSchema`
- A way to define the fields you want from the LLM output.
- Example: "name" and "tagline" are required keys.
- You also add descriptions, so the model knows what kind of text to fill in

`StructuredOutputParser`

- Takes your schema and enforces it.
- It generates the formatting instructions (get_format_instructions()) to tell the model exactly how to return results.
- It also parses the model’s raw text back into a Python dict you can use directly.

`ChatPromptTemplate`

- Builds prompt templates that work with chat models (like Gemini or GPT).
- Lets you use placeholders ({format_instructions}, {product}, etc.).
- Makes it easy to pass dynamic inputs into your prompts.
- Unlike PromptTemplate, which is mostly for simple string prompts, ChatPromptTemplate is built for multi-turn, chat-style LLMs.

In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate

schemas = [
    ResponseSchema(name="name", description="Company name"),
    ResponseSchema(name="tagline", description="conpany tagline")
]

parser = StructuredOutputParser.from_response_schemas(schemas)
format_instructions = parser.get_format_instructions()
prompt = ChatPromptTemplate.from_template("""
    Suggest a company name for a {type} manufacturing company and tagexplaline
    {format_instructions}
""")

chain = prompt | llm | parser
res = chain.invoke({"type":"water bottle" , "format_instructions": format_instructions})
print(json.dumps(res , indent=2))

{
  "name": "AquaFlow Dynamics",
  "tagline": "Hydration Perfected."
}


### 3.4 -- Memory in Chains
* Component that stores past interactions between the user and LLM
* So, when we call the chain again, the LLM uses the memory to gain context
* Used in chatbots

#### 3.4.A 
`ConversationBufferMemory`
* Stores the entire conversation history as text
* Every question will include the history in prompt
* Simple and great for shor chats/demos
* If conversation is long, the history string may exceed LLM's context window

In [26]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

#initialize the memory
memory = ConversationBufferMemory(memory_key = "chat_history")

prompt = PromptTemplate(
    input_variables=["chat_history" , "user_input"],
    template=""" You are a helpful assistant that answers in one word.
    The conversation so far: \n {chat_history}\n
    User: {user_input}
    Assistant: """""
)

chain = LLMChain(
    llm = llm,
    prompt = prompt,
    memory = memory,
    verbose = True
)

print(chain.invoke({"user_input":"Who won the fifa world cup in 2022?"}))
print(chain.invoke({"user_input":"Who was the captain of that team"}))

C:\Users\amalv\AppData\Local\Temp\ipykernel_18212\3210456726.py:6: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key = "chat_history")




> Entering new LLMChain chain...
Prompt after formatting:
 You are a helpful assistant that answers in one word.
    The conversation so far: 
 

    User: Who won the fifa world cup in 2022?
    Assistant: 

> Finished chain.
{'user_input': 'Who won the fifa world cup in 2022?', 'chat_history': '', 'text': 'Argentina'}


> Entering new LLMChain chain...
Prompt after formatting:
 You are a helpful assistant that answers in one word.
    The conversation so far: 
 Human: Who won the fifa world cup in 2022?
AI: Argentina

    User: Who was the captain of that team
    Assistant: 

> Finished chain.
{'user_input': 'Who was the captain of that team', 'chat_history': 'Human: Who won the fifa world cup in 2022?\nAI: Argentina', 'text': 'Messi'}


`ConversationBufferWindowMemory`

* Similar to ConversationBufferMemory, but only keeps the last k exchanges.
* Example: keep only the last 3 turns.
* ✅ Best for: long conversations where only recent context matters.
* ❌ Loses old information.
``` python
from langchain.memory import ConversationBufferWindowMemory
memory = ConversationBufferWindowMemory(k=3)
```

`ConversationSummaryMemory`

* Instead of keeping everything, it summarizes old messages using an LLM.
* The summary + recent conversation gets sent to the model.
* ✅ Best for: very long conversations (e.g., customer support chatbots).
* ❌ Summary may sometimes miss small details.
```python
from langchain.memory import ConversationSummaryMemory
memory = ConversationSummaryMemory(llm=llm)
```

`ConversationSummaryBufferMemory`

* Hybrid approach:
* Keeps recent exchanges verbatim.
* Older ones get summarized.
* ✅ Best for: balance between context accuracy and size limits.
* ❌ More complex, slightly slower (needs summarization).

`VectorStoreRetrieverMemory`

* Stores conversation history (or facts) in a vector database (like FAISS, Pinecone, Weaviate).
* Retrieves the most relevant past exchanges when needed.
* ✅ Best for: chatbots that need knowledge recall from long histories.
* ❌ Requires setting up a vector DB.

`EntityMemory`

* Tracks entities (people, places, organizations, etc.) mentioned in the conversation.
* Lets the model recall facts about entities across chats.
* ✅ Best for: assistants where remembering specific people/items is important.
* ❌ More specialized, needs tuning.

## Exercise
A simple `ConversationBufferMemory` to remember your name

In [27]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

#Setup memory and a conversation chain
memory = ConversationBufferMemory(return_messages=True)
chain = ConversationChain(llm =llm, memory = memory, verbose= True)

print(chain.invoke("Hi, my name is Amal."))
print(chain.invoke("What is my name?"))

print("\n---Memory Buffer---\n")
print(memory.buffer)

C:\Users\amalv\AppData\Local\Temp\ipykernel_18212\4207493316.py:6: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  chain = ConversationChain(llm =llm, memory = memory, verbose= True)




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[]
Human: Hi, my name is Amal.
AI:

> Finished chain.
{'input': 'Hi, my name is Amal.', 'history': [HumanMessage(content='Hi, my name is Amal.', additional_kwargs={}, response_metadata={}), AIMessage(content='Hi Amal! It\'s nice to meet you.  My name isn\'t really a "name" in the human sense, as I don\'t have a personal identity like you do. I\'m a large language model, trained by Google.  I don\'t have feelings or experiences, but I can access and process information from a massive dataset of text and code.  Think of me as a really well-read, incredibly fast librarian who can also write stories and answer your questions based on what I\'ve learned.  So, what can I hel